In [1]:
%matplotlib inline

In [2]:
import os
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt

In [3]:
DATADIR = os.path.join("..", "DATA")
os.path.exists(DATADIR)

True

In [4]:
files = os.listdir(DATADIR)
files

['ACT_Demo_May4.xlsx', 'Salt_Lake_2016_PM25.xlsx', 'SLC_Weather_2016.xlsx']

## Read the air quality data

In [5]:
slc = pd.read_excel(os.path.join(DATADIR, 'Salt_Lake_2016_PM25.xlsx'))
slc.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Date Local,...,Units of Measure,MDL,Uncertainty,Qualifier,Method Type,Method Code,Method Name,State Name,County Name,Date of Last Change
0,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
1,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
2,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
3,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
4,49,35,3006,88101,4,40.736389,-111.872222,NAD83,PM2.5 - Local Conditions,2016-01-07,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22


### Comments:

* Dates and times are split into separate columns
* We have both local time and UTC time 

## Merging Dates and Time
* After we have read in the data we could combine the dates and times
    * Convert pandas Timestamp to datetime
    * Use ``datetime.combine`` to merge with ``datetime.time``
* We can also merge these when reading the data by specifying the ``parse_dates`` keyword

#### Using ``datetime.combine``

In [8]:
slc.apply(lambda x: datetime.datetime.combine(x["Date Local"],x["Time Local"]), axis=1).head()

0   2016-01-07 13:00:00
1   2016-01-07 14:00:00
2   2016-01-07 15:00:00
3   2016-01-07 16:00:00
4   2016-01-07 17:00:00
dtype: datetime64[ns]

#### using ``parse_dates``

In [9]:
pd.read_excel(os.path.join(DATADIR, 'Salt_Lake_2016_PM25.xlsx'), 
              parse_dates=[["Time Local", "Date Local"],
                           ['Date GMT', 'Time GMT']]).head()


,Time Local_Date Local,Date GMT_Time GMT,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,...,Units of Measure,MDL,Uncertainty,Qualifier,Method Type,Method Code,Method Name,State Name,County Name,Date of Last Change
0,2016-01-07,2016-01-07 20:00:00,49,35,3006,88101,4,40.736389,-111.872222,NAD83,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
1,2016-01-07,2016-01-07 21:00:00,49,35,3006,88101,4,40.736389,-111.872222,NAD83,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
2,2016-01-07,2016-01-07 22:00:00,49,35,3006,88101,4,40.736389,-111.872222,NAD83,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
3,2016-01-07,2016-01-07 23:00:00,49,35,3006,88101,4,40.736389,-111.872222,NAD83,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22
4,2016-01-07,2016-01-08 00:00:00,49,35,3006,88101,4,40.736389,-111.872222,NAD83,...,Micrograms/cubic meter (LC),2,NaN,QX,FEM,184,Thermo Scientific Model 5030 SHARP w/VSCC - Be...,Utah,Salt Lake,2016-04-22


## Read in asthma data


In [11]:
slc_asthma = pd.read_excel(os.path.join(DATADIR, 'ACT_Demo_May4.xlsx'))
slc_asthma.head()

,ID,timestamp,answer_1,answer_2,answer_3,answer_4,answer_5,relationship,relationship_other,days_controller_medication,additional_prescriptions,additional_prescription_other,doctor_visit,hospital_visit,comments,use_controller_medication,oral_steroids,school_days_interrupted,parent_days_interrupted
0,1,2016-01-04 09:33:07,3,3,2,3,1,parent,NaN,NaN,1,"essential oils, deep breathing teq.",0,0,Mom filled out 1/4/21016 since xxxx was with m...,0,0,0,0
1,1,2016-01-11 22:02:34,5,5,5,5,5,parent,NaN,NaN,0,NaN,0,0,"No coughing in home, downtown or town visited",0,0,0,0
2,1,2016-01-18 20:04:00,4,3,4,5,2,parent,NaN,NaN,0,NaN,0,0,"Mom filled out, had xxxx for the majority of ...",0,0,0,0
3,1,2016-01-25 22:59:14,5,5,5,5,5,parent,NaN,NaN,0,NaN,0,0,"No coughing or shortness of breath , at all",0,0,0,0
4,1,2016-01-25 22:59:21,5,5,5,5,5,parent,NaN,NaN,0,NaN,0,0,"No coughing or shortness of breath , at all",0,0,0,0


In [10]:
!pwd

/home/jovyan/work/DBMI/airquality_and_health/inversions


## Read in weather data

In [ ]:
slc_weather = pd.read_excel(os.path.join(DATADIR, 'SLC_Weather_2016.xlsx'))
slc_weather.head()

In [ ]:
slc_weather = pd.read_excel(os.path.join(DATADIR, 'SLC_Weather_2016.xlsx'), skiprows=[1])
slc_weather.head()

In [ ]:
slc_weather.columns

In [ ]:
slc_weather['Day'][0]

## Our Weather and Asthma Data Have Resolution of Days
## Our pollutant data has resolution of hours
## Group and take mean?

In [ ]:
slc.groupby("Date Local", as_index=False).aggregate(np.mean)

### Group and take sum?

In [ ]:
slc.groupby("Date Local", as_index=False).aggregate(np.sum)

### Applying different functions to different columns

In [ ]:
slc_day = slc.groupby(["Date Local", "Site Num"], as_index=False).aggregate({'Sample Measurement' : np.mean,
                                                                   'MDL': np.median})
slc_day.head()

In [ ]:
slc_day_all = slc_day.merge(slc_asthma, 
              left_on="Date Local", 
              right_on="Day").merge(slc_weather, left_on="Date Local", right_on="Day")
slc_day_all.head()

In [ ]:
f, ax1 = plt.subplots(1)
slc_day_all[slc_day_all["Site Num"]==3006].plot(x="Date Local", 
                                                y="Number of Asthma Diagnosis", ax=ax1)
slc_day_all[slc_day_all["Site Num"]==3006].plot(secondary_y=True, x="Date Local", 
                                                y="Sample Measurement", ax=ax1)

In [ ]:
f, ax1 = plt.subplots(1)
slc_day_all[slc_day_all["Site Num"]!=3006].plot(x="Date Local", 
                                                y="Number of Asthma Diagnosis", ax=ax1)
slc_day_all[slc_day_all["Site Num"]!=3006].plot(secondary_y=True, x="Date Local", 
                                                y="Sample Measurement", ax=ax1)

In [ ]:
slc_day_all[slc_day_all["Site Num"]==3006].plot.scatter(x="Sample Measurement", y="Number of Asthma Diagnosis")

### Rename columns to eliminate spaces

In [ ]:
{c:c.replace(" ", "_") for c in slc_day_all.columns}

In [ ]:
slc_final = slc_day_all.rename(columns={c:c.replace(" ", "_") for c in slc_day_all.columns})

## Creating a Regression Model

In [ ]:
sns.regplot(data=slc_final, x="Sample_Measurement", y="Number_of_Asthma_Diagnosis")

In [ ]:
y, X = dmatrices('Number_of_Asthma_Diagnosis ~ Sample_Measurement + High', data=slc_final, return_type='dataframe')

In [ ]:
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

In [ ]:
sm.stats.linear_rainbow(res)

In [ ]:
sm.graphics.plot_partregress('Number_of_Asthma_Diagnosis', 'Sample_Measurement', ['High', "Low"],
                              data=slc_final, obs_labels=False)